# Tutorial: Fan-Out/Fan-In Pattern with Queue Workers

**Category**: Concurrency
**Difficulty**: Advanced
**Time**: 25-35 minutes

## Problem Statement

In production systems, you often need to distribute work across multiple concurrent workers for parallelism, then collect and aggregate results. For example, processing user uploads (resize images, extract metadata, scan for malware) across a worker pool, or distributing API requests to fetch data from multiple endpoints concurrently. The challenge is coordinating work distribution (fan-out), managing worker lifecycles, and collecting results efficiently (fan-in) while handling errors gracefully.

Consider a data pipeline that needs to process 10,000 records from a database. Sequential processing would take hours. You need to distribute records across 10 workers, each processing batches concurrently. But how do you feed work to workers? How do you know when all work is complete? How do you handle workers that crash? How do you collect results without blocking producers?

**Why This Matters**:
- **Throughput**: Worker pools maximize CPU and I/O utilization (10-100x speedup for parallelizable tasks)
- **Reliability**: Proper queue patterns prevent work loss, handle backpressure, and isolate worker failures
- **Resource Management**: Bounded queues prevent memory exhaustion from fast producers overwhelming slow consumers

**What You'll Build**:
A production-ready fan-out/fan-in system using lionherd-core's `Queue` and `create_task_group` that distributes work across multiple concurrent workers, collects results efficiently, and handles errors gracefully with proper lifecycle management.

## Prerequisites

**Prior Knowledge**:
- Python async/await fundamentals
- Producer-consumer pattern basics
- Exception handling in concurrent contexts
- Task groups and structured concurrency

**Required Packages**:
```bash
pip install lionherd-core  # >=0.1.0
```

**Optional Reading**:
- [API Reference: Concurrency Primitives](../../docs/api/libs/concurrency/primitives.md)
- [API Reference: Task Groups](../../docs/api/libs/concurrency/task.md)
- [Reference Notebook: Concurrency Primitives](../references/concurrency_primitives.ipynb)

In [ ]:
# Standard library
import asyncio
from collections import defaultdict
from dataclasses import dataclass, field
from enum import Enum
from typing import Any, Callable, Generic, TypeVar

# lionherd-core
from lionherd_core.libs.concurrency import (
    Event,
    Queue,
    create_task_group,
    current_time,
    sleep,
)

T = TypeVar("T")
R = TypeVar("R")

## Solution Overview

We'll implement a fan-out/fan-in system using queues and worker pools:

1. **Work Queue**: Producers add tasks to a shared queue (fan-out)
2. **Worker Pool**: N workers concurrently consume and process tasks
3. **Result Collection**: Workers send results to a result queue (fan-in)
4. **Lifecycle Management**: Coordinate worker startup, shutdown, and completion signaling

**Key lionherd-core Components**:
- `Queue`: Thread-safe async queue with backpressure (maxsize)
- `create_task_group`: Structured concurrency for worker lifecycle management
- `Event`: Signal completion when all work is done
- `get_nowait`: Non-blocking queue read for graceful shutdown

**Flow**:
```
Producer → [Work Queue] → Worker 1 → [Result Queue] → Result Collector
                       ↓→ Worker 2 →↓
                       ↓→ Worker 3 →↓
                       ↓→ Worker N →↓
```

**Expected Outcome**: Process tasks in parallel across worker pool, collect all results, and shut down gracefully when complete.

In [ ]:
# Quick Start: Worker Pool in 30 Seconds

from lionherd_core.libs.concurrency import Queue, create_task_group, sleep
import asyncio

async def worker(worker_id: int, queue: Queue, results: list):
    """Pull tasks from queue and process."""
    while True:
        item = await queue.get()
        if item is None:  # Shutdown signal
            await queue.put(None)  # Pass to other workers
            break
        
        # Process
        await sleep(0.05)
        results.append(f"Worker {worker_id}: {item}")

# Try it:
work_queue = Queue.with_maxsize(10)
results = []

async with create_task_group() as tg:
    # Start 3 workers
    for i in range(3):
        tg.start_soon(worker, i, work_queue, results)
    
    # Add work
    for i in range(10):
        await work_queue.put(f"task-{i}")
    
    # Shutdown
    await work_queue.put(None)

print(f"Processed {len(results)} items with 3 workers")
for r in results[:5]:
    print(f"  {r}")

# 👇 Now read below to understand production-ready worker pools

### Step 1: Define Task and Result Data Structures

We need to represent work items (tasks) and their outcomes (results). Tasks carry data and processing logic. Results track success/failure and processing metadata.

**Why Structured Data**: Type-safe tasks prevent runtime errors. Result metadata enables debugging, performance analysis, and error recovery.

In [ ]:
@dataclass
class WorkItem(Generic[T]):
    """A unit of work to process."""

    id: str
    data: T
    priority: int = 0  # For priority queue variants
    metadata: dict[str, Any] = field(default_factory=dict)

    def __repr__(self) -> str:
        return f"WorkItem(id={self.id!r}, data={self.data!r})"


class ResultStatus(Enum):
    """Processing outcome."""

    SUCCESS = "success"
    FAILED = "failed"


@dataclass
class WorkResult(Generic[R]):
    """Result of processing a work item."""

    work_id: str
    status: ResultStatus
    result: R | None = None
    error: Exception | None = None
    worker_id: int = 0
    duration: float = 0.0  # Processing time in seconds

    def __repr__(self) -> str:
        return f"WorkResult(id={self.work_id!r}, status={self.status.value}, worker={self.worker_id}, duration={self.duration:.3f}s)"


# Example: Create work item and result
work = WorkItem(id="task-001", data={"value": 42})
result = WorkResult(
    work_id="task-001",
    status=ResultStatus.SUCCESS,
    result="processed",
    worker_id=1,
    duration=0.15,
)

print(f"Work: {work}")
print(f"Result: {result}")

**Notes**:
- **Generic Types**: `WorkItem[T]` and `WorkResult[R]` provide type safety - the compiler catches mismatches
- **Worker ID**: Tracking which worker processed each task enables debugging and load balancing analysis
- **Duration**: Essential for identifying slow tasks and performance bottlenecks
- **Metadata**: Store retry count, timestamps, or custom context without polluting the core structure

### Step 2: Implement Basic Worker Pattern

A worker continuously pulls tasks from the work queue, processes them, and sends results to the result queue. Workers must handle the "poison pill" pattern for graceful shutdown - a sentinel value (None) signals "no more work."

**Why Poison Pill**: Without a shutdown signal, workers would block forever on `queue.get()`. The poison pill pattern ensures deterministic shutdown.

In [ ]:
async def worker(
    worker_id: int,
    work_queue: Queue[WorkItem[T] | None],
    result_queue: Queue[WorkResult[R]],
    process_func: Callable[[T], R],
) -> None:
    """Worker that processes items from queue.

    Args:
        worker_id: Unique worker identifier
        work_queue: Queue to pull work from (None = shutdown signal)
        result_queue: Queue to send results to
        process_func: Async function to process work item data
    """
    print(f"Worker {worker_id}: Started")

    while True:
        # Get work item (blocks until available)
        work_item = await work_queue.get()

        # Poison pill - shutdown signal
        if work_item is None:
            print(f"Worker {worker_id}: Received shutdown signal")
            break

        # Process work item
        start_time = current_time()
        try:
            result = await process_func(work_item.data)
            duration = current_time() - start_time

            await result_queue.put(
                WorkResult(
                    work_id=work_item.id,
                    status=ResultStatus.SUCCESS,
                    result=result,
                    worker_id=worker_id,
                    duration=duration,
                )
            )
        except Exception as e:
            duration = current_time() - start_time
            await result_queue.put(
                WorkResult(
                    work_id=work_item.id,
                    status=ResultStatus.FAILED,
                    error=e,
                    worker_id=worker_id,
                    duration=duration,
                )
            )

    print(f"Worker {worker_id}: Shutdown complete")


# Test: Simple worker pool
async def simple_process(value: int) -> int:
    """Simulate processing - square the value."""
    await sleep(0.1)  # Simulate work
    return value * value


async def test_basic_worker():
    work_queue = Queue[WorkItem[int] | None].with_maxsize(10)
    result_queue = Queue[WorkResult[int]].with_maxsize(10)

    # Start worker
    async with create_task_group() as tg:
        tg.start_soon(worker, 1, work_queue, result_queue, simple_process)

        # Add work
        for i in range(5):
            await work_queue.put(WorkItem(id=f"task-{i}", data=i))

        # Send shutdown signal
        await work_queue.put(None)

    # Collect results (worker has exited, so all results are available)
    results = []
    try:
        while True:
            results.append(result_queue.get_nowait())
    except Exception:  # WouldBlock exception when queue empty
        pass

    print(f"\nCollected {len(results)} results:")
    for r in results:
        print(f"  {r}")


await test_basic_worker()

**Notes**:
- **Blocking `get()`**: Workers block on `queue.get()` until work arrives - this is efficient (no busy-waiting)
- **Poison Pill**: Sending `None` is the standard pattern for shutdown. Each worker consumes one poison pill.
- **Error Handling**: Worker failures don't crash the worker - errors are captured in `WorkResult` and processing continues
- **get_nowait()**: Non-blocking read used for result collection after workers exit (throws exception when empty)

### Step 3: Add Worker Pool Management

A single worker is limiting. We need a pool of N workers for parallelism. This requires coordinating worker startup, shutdown (N poison pills for N workers), and tracking when all work is complete.

**Why N Poison Pills**: Each worker consumes one poison pill. If you only send one, only one worker shuts down - the rest block forever.

In [ ]:
@dataclass
class WorkerPoolConfig:
    """Configuration for worker pool."""

    num_workers: int = 4
    work_queue_size: int = 100
    result_queue_size: int = 100


async def run_worker_pool(
    work_items: list[WorkItem[T]],
    process_func: Callable[[T], R],
    config: WorkerPoolConfig | None = None,
) -> list[WorkResult[R]]:
    """Run worker pool to process work items.

    Args:
        work_items: List of work to process
        process_func: Async function to process each work item's data
        config: Worker pool configuration

    Returns:
        List of work results (one per work item)
    """
    config = config or WorkerPoolConfig()

    # Create queues
    work_queue = Queue[WorkItem[T] | None].with_maxsize(config.work_queue_size)
    result_queue = Queue[WorkResult[R]].with_maxsize(config.result_queue_size)

    async with create_task_group() as tg:
        # Start workers
        for i in range(config.num_workers):
            tg.start_soon(worker, i, work_queue, result_queue, process_func)

        # Producer: Add all work items
        for item in work_items:
            await work_queue.put(item)

        # Send poison pills (one per worker)
        for _ in range(config.num_workers):
            await work_queue.put(None)

    # All workers have exited - collect results
    results = []
    try:
        while True:
            results.append(result_queue.get_nowait())
    except Exception:  # Queue empty
        pass

    return results


# Test: Process 20 items with 4 workers
async def test_worker_pool():
    work_items = [WorkItem(id=f"task-{i}", data=i) for i in range(20)]

    config = WorkerPoolConfig(num_workers=4)
    results = await run_worker_pool(work_items, simple_process, config)

    print(f"\nProcessed {len(results)} items with {config.num_workers} workers:")

    # Group results by worker
    worker_counts = defaultdict(int)
    for r in results:
        worker_counts[r.worker_id] += 1

    print("\nWork distribution:")
    for worker_id, count in sorted(worker_counts.items()):
        print(f"  Worker {worker_id}: {count} tasks")

    # Show sample results
    print("\nSample results:")
    for r in results[:5]:
        print(f"  {r}")


await test_worker_pool()

**Notes**:
- **Task Group**: Using `create_task_group()` ensures all workers complete before proceeding to result collection
- **N Poison Pills**: Critical - you must send exactly as many poison pills as workers, or some workers hang forever
- **Work Distribution**: Notice how work is distributed across workers (roughly equal, but not perfectly balanced)
- **Queue Sizing**: `maxsize` provides backpressure - if producer is faster than workers, `put()` blocks

### Step 4: Add Progress Tracking and Statistics

Production systems need observability: how many tasks completed, failed, average processing time, per-worker throughput. We'll add comprehensive statistics collection.

**Why Statistics**: Without metrics, you can't answer "Are workers balanced?" or "Is worker 3 slower than others?" Production needs visibility.

In [ ]:
@dataclass
class PoolStatistics:
    """Statistics from worker pool execution."""

    total_items: int
    successful: int
    failed: int
    total_duration: float
    avg_task_duration: float
    worker_counts: dict[int, int]  # Tasks per worker
    worker_durations: dict[int, float]  # Total time per worker

    def __repr__(self) -> str:
        return (
            f"PoolStatistics(\n"
            f"  total={self.total_items}, success={self.successful}, failed={self.failed}\n"
            f"  total_duration={self.total_duration:.3f}s, avg_task={self.avg_task_duration:.3f}s\n"
            f"  workers={len(self.worker_counts)}\n"
            f")"
        )


async def run_worker_pool_with_stats(
    work_items: list[WorkItem[T]],
    process_func: Callable[[T], R],
    config: WorkerPoolConfig | None = None,
) -> tuple[list[WorkResult[R]], PoolStatistics]:
    """Run worker pool with comprehensive statistics.

    Args:
        work_items: List of work to process
        process_func: Async function to process each work item
        config: Worker pool configuration

    Returns:
        (results, statistics)
    """
    config = config or WorkerPoolConfig()
    start_time = current_time()

    # Create queues
    work_queue = Queue[WorkItem[T] | None].with_maxsize(config.work_queue_size)
    result_queue = Queue[WorkResult[R]].with_maxsize(config.result_queue_size)

    async with create_task_group() as tg:
        # Start workers
        for i in range(config.num_workers):
            tg.start_soon(worker, i, work_queue, result_queue, process_func)

        # Producer: Add all work items
        for item in work_items:
            await work_queue.put(item)

        # Send poison pills
        for _ in range(config.num_workers):
            await work_queue.put(None)

    # Collect results
    results = []
    try:
        while True:
            results.append(result_queue.get_nowait())
    except Exception:
        pass

    # Calculate statistics
    total_duration = current_time() - start_time
    successful = sum(1 for r in results if r.status == ResultStatus.SUCCESS)
    failed = sum(1 for r in results if r.status == ResultStatus.FAILED)

    worker_counts = defaultdict(int)
    worker_durations = defaultdict(float)
    for r in results:
        worker_counts[r.worker_id] += 1
        worker_durations[r.worker_id] += r.duration

    avg_duration = sum(r.duration for r in results) / len(results) if results else 0.0

    stats = PoolStatistics(
        total_items=len(work_items),
        successful=successful,
        failed=failed,
        total_duration=total_duration,
        avg_task_duration=avg_duration,
        worker_counts=dict(worker_counts),
        worker_durations=dict(worker_durations),
    )

    return results, stats


# Test: Process with statistics
async def variable_work(value: int) -> int:
    """Simulate variable processing time."""
    # Simulate slower processing for higher values
    await sleep(0.05 + (value % 3) * 0.05)
    return value * 2


async def test_with_stats():
    work_items = [WorkItem(id=f"task-{i}", data=i) for i in range(50)]
    config = WorkerPoolConfig(num_workers=5)

    results, stats = await run_worker_pool_with_stats(work_items, variable_work, config)

    print(stats)
    print("\nPer-worker statistics:")
    for worker_id in sorted(stats.worker_counts.keys()):
        count = stats.worker_counts[worker_id]
        duration = stats.worker_durations[worker_id]
        avg = duration / count if count > 0 else 0
        print(
            f"  Worker {worker_id}: {count} tasks, {duration:.3f}s total, {avg:.3f}s avg"
        )


await test_with_stats()

**Notes**:
- **Total vs Task Duration**: `total_duration` is wall-clock time (includes parallelism), `avg_task_duration` is per-task processing time
- **Worker Statistics**: Per-worker metrics reveal load imbalance or slow workers (hardware issues, network problems)
- **Production Monitoring**: Export these stats to Prometheus/DataDog for alerting and capacity planning
- **Load Balancing**: Notice work isn't perfectly balanced - this is expected with FIFO queue (priority queues can improve this)

### Step 5: Add Result Streaming and Early Termination

For long-running pools, waiting for all results before processing is inefficient. We want to stream results as they arrive (async iteration) and support early termination (stop after N results or first error).

**Why Streaming**: Processing results as they arrive enables real-time updates, reduces memory usage, and allows early exit patterns.

In [ ]:
async def stream_worker_pool(
    work_items: list[WorkItem[T]],
    process_func: Callable[[T], R],
    config: WorkerPoolConfig | None = None,
):
    """Stream results from worker pool as they arrive.

    Args:
        work_items: List of work to process
        process_func: Async function to process work items
        config: Worker pool configuration

    Yields:
        WorkResult objects as they complete
    """
    config = config or WorkerPoolConfig()

    # Create queues
    work_queue = Queue[WorkItem[T] | None].with_maxsize(config.work_queue_size)
    result_queue = Queue[WorkResult[R] | None].with_maxsize(config.result_queue_size)

    # Modified worker that signals completion
    async def streaming_worker(
        worker_id: int,
        work_queue: Queue[WorkItem[T] | None],
        result_queue: Queue[WorkResult[R] | None],
        process_func: Callable[[T], R],
    ) -> None:
        while True:
            work_item = await work_queue.get()
            if work_item is None:
                # Signal this worker is done
                await result_queue.put(None)
                break

            start_time = current_time()
            try:
                result = await process_func(work_item.data)
                duration = current_time() - start_time
                await result_queue.put(
                    WorkResult(
                        work_id=work_item.id,
                        status=ResultStatus.SUCCESS,
                        result=result,
                        worker_id=worker_id,
                        duration=duration,
                    )
                )
            except Exception as e:
                duration = current_time() - start_time
                await result_queue.put(
                    WorkResult(
                        work_id=work_item.id,
                        status=ResultStatus.FAILED,
                        error=e,
                        worker_id=worker_id,
                        duration=duration,
                    )
                )

    # Start workers and producer in background
    async with create_task_group() as tg:
        # Start workers
        for i in range(config.num_workers):
            tg.start_soon(streaming_worker, i, work_queue, result_queue, process_func)

        # Producer task
        async def producer():
            for item in work_items:
                await work_queue.put(item)
            # Send poison pills
            for _ in range(config.num_workers):
                await work_queue.put(None)

        tg.start_soon(producer)

        # Consumer: Stream results as they arrive
        workers_done = 0
        while workers_done < config.num_workers:
            result = await result_queue.get()

            if result is None:
                workers_done += 1
                continue

            yield result


# Test: Stream results and stop early
async def test_streaming():
    work_items = [WorkItem(id=f"task-{i}", data=i) for i in range(100)]
    config = WorkerPoolConfig(num_workers=4)

    print("Streaming results (first 10 only):")
    count = 0
    async for result in stream_worker_pool(work_items, simple_process, config):
        print(f"  {result}")
        count += 1
        if count >= 10:
            print("  ... (stopping early, workers still running)")
            break


await test_streaming()

**Notes**:
- **Worker Completion Signal**: Workers send `None` to result queue when done - the consumer counts these to know when all workers have exited
- **Early Termination**: Breaking from the async for loop stops consuming results, but workers continue running in the background
- **Backpressure**: If consumer stops reading, workers will block on `result_queue.put()` when queue fills (controlled by `maxsize`)
- **Memory Efficiency**: Streaming prevents loading all results into memory - critical for large workloads

## Complete Working Example

Here's the full production-ready implementation combining all steps. Copy-paste this into your project.

**Features**:
- ✅ Configurable worker pool with fan-out/fan-in pattern
- ✅ Comprehensive statistics and progress tracking
- ✅ Result streaming for real-time processing
- ✅ Graceful shutdown with poison pill pattern
- ✅ Error handling and per-worker metrics
- ✅ Type-safe with Generic work items and results

In [ ]:
"""Complete production-ready fan-out/fan-in worker pool.

Copy this entire cell into your project and adjust configuration.
"""

# Standard library
from collections import defaultdict
from dataclasses import dataclass, field
from enum import Enum
from typing import Any, Callable, Generic, TypeVar

# lionherd-core
from lionherd_core.libs.concurrency import Queue, create_task_group, current_time, sleep

T = TypeVar("T")
R = TypeVar("R")


@dataclass
class WorkItem(Generic[T]):
    """Unit of work to process."""

    id: str
    data: T
    priority: int = 0
    metadata: dict[str, Any] = field(default_factory=dict)


class ResultStatus(Enum):
    SUCCESS = "success"
    FAILED = "failed"


@dataclass
class WorkResult(Generic[R]):
    """Result of processing work item."""

    work_id: str
    status: ResultStatus
    result: R | None = None
    error: Exception | None = None
    worker_id: int = 0
    duration: float = 0.0


@dataclass
class PoolStatistics:
    """Worker pool execution statistics."""

    total_items: int
    successful: int
    failed: int
    total_duration: float
    avg_task_duration: float
    worker_counts: dict[int, int]
    worker_durations: dict[int, float]


@dataclass
class WorkerPoolConfig:
    """Worker pool configuration."""

    num_workers: int = 4
    work_queue_size: int = 100
    result_queue_size: int = 100


async def worker(
    worker_id: int,
    work_queue: Queue[WorkItem[T] | None],
    result_queue: Queue[WorkResult[R]],
    process_func: Callable[[T], R],
) -> None:
    """Worker that processes items from queue."""
    while True:
        work_item = await work_queue.get()
        if work_item is None:  # Poison pill
            break

        start_time = current_time()
        try:
            result = await process_func(work_item.data)
            duration = current_time() - start_time
            await result_queue.put(
                WorkResult(
                    work_id=work_item.id,
                    status=ResultStatus.SUCCESS,
                    result=result,
                    worker_id=worker_id,
                    duration=duration,
                )
            )
        except Exception as e:
            duration = current_time() - start_time
            await result_queue.put(
                WorkResult(
                    work_id=work_item.id,
                    status=ResultStatus.FAILED,
                    error=e,
                    worker_id=worker_id,
                    duration=duration,
                )
            )


async def run_worker_pool(
    work_items: list[WorkItem[T]],
    process_func: Callable[[T], R],
    config: WorkerPoolConfig | None = None,
) -> tuple[list[WorkResult[R]], PoolStatistics]:
    """Run worker pool with fan-out/fan-in pattern.

    Args:
        work_items: List of work to process
        process_func: Async function to process work item data
        config: Worker pool configuration

    Returns:
        (results, statistics)
    """
    config = config or WorkerPoolConfig()
    start_time = current_time()

    work_queue = Queue[WorkItem[T] | None].with_maxsize(config.work_queue_size)
    result_queue = Queue[WorkResult[R]].with_maxsize(config.result_queue_size)

    async with create_task_group() as tg:
        # Start workers
        for i in range(config.num_workers):
            tg.start_soon(worker, i, work_queue, result_queue, process_func)

        # Producer: add work + poison pills
        for item in work_items:
            await work_queue.put(item)
        for _ in range(config.num_workers):
            await work_queue.put(None)

    # Collect results
    results = []
    try:
        while True:
            results.append(result_queue.get_nowait())
    except Exception:
        pass

    # Statistics
    total_duration = current_time() - start_time
    successful = sum(1 for r in results if r.status == ResultStatus.SUCCESS)
    failed = sum(1 for r in results if r.status == ResultStatus.FAILED)

    worker_counts = defaultdict(int)
    worker_durations = defaultdict(float)
    for r in results:
        worker_counts[r.worker_id] += 1
        worker_durations[r.worker_id] += r.duration

    avg_duration = sum(r.duration for r in results) / len(results) if results else 0.0

    stats = PoolStatistics(
        total_items=len(work_items),
        successful=successful,
        failed=failed,
        total_duration=total_duration,
        avg_task_duration=avg_duration,
        worker_counts=dict(worker_counts),
        worker_durations=dict(worker_durations),
    )

    return results, stats


# Example usage
async def main():
    """Example: Process image batch with worker pool."""

    # Simulate image processing
    async def process_image(image_path: str) -> dict[str, Any]:
        await sleep(0.1)  # Simulate processing
        return {
            "path": image_path,
            "width": 1920,
            "height": 1080,
            "format": "JPEG",
        }

    # Create work items
    work_items = [
        WorkItem(id=f"image-{i}", data=f"/uploads/image_{i}.jpg") for i in range(100)
    ]

    # Process with 8 workers
    config = WorkerPoolConfig(num_workers=8)
    results, stats = await run_worker_pool(work_items, process_image, config)

    # Report
    print(f"Processed {stats.successful}/{stats.total_items} images")
    print(f"Total time: {stats.total_duration:.2f}s")
    print(f"Avg task: {stats.avg_task_duration:.3f}s")
    print(f"\nPer-worker breakdown:")
    for worker_id in sorted(stats.worker_counts.keys()):
        count = stats.worker_counts[worker_id]
        print(f"  Worker {worker_id}: {count} images")


# Run example
await main()

## Common Issues

**Worker Crash**:
- Symptom: Individual worker throws unhandled exception
- Fix: Wrap worker loop in try-except, log and restart

**Queue Deadlock**:
- Symptom: Producer blocks waiting for queue space, workers waiting for items
- Fix: Ensure queue_size >= num_workers × 2, monitor queue depth

**Memory Leak**:
- Symptom: Results list grows unbounded for large batches
- Fix: Stream results to disk or process incrementally

For production patterns (monitoring, error recovery, dynamic scaling), see [lionherd-core Production Guide](https://github.com/khive-ai/lionherd-core/docs/production/worker_pools.md).

## Variation: Priority-Based Work Distribution

**When to Use**: Work items have different priorities (critical vs nice-to-have)

**Pattern**:
```python
from lionherd_core.libs.concurrency import PriorityQueue
from dataclasses import dataclass, field

@dataclass(order=True)
class PriorityWorkItem(Generic[T]):
    priority: int  # Lower = higher priority
    item: WorkItem[T] = field(compare=False)

# Use PriorityQueue instead of Queue
work_queue = PriorityQueue.with_maxsize(100)

# Producer adds items with priority
for item in work_items:
    await work_queue.put(PriorityWorkItem(priority=item.priority, item=item))

# Workers extract: priority, item = await work_queue.get()
```

**Trade-offs**:
- ✅ High-priority items processed first
- ✅ Good for SLA-tiered workloads
- ❌ Low-priority items may starve
- ❌ Priority queue has ~2x overhead vs FIFO

For additional variations (Dynamic Scaling, Batched Results), see [lionherd-core examples](https://github.com/khive-ai/lionherd-core/examples/worker_pool_variations.py).

## Summary

**What You Accomplished**:
- ✅ Built production-ready fan-out/fan-in worker pool using `Queue` and `create_task_group`
- ✅ Implemented graceful shutdown with poison pill pattern
- ✅ Added comprehensive statistics and per-worker metrics
- ✅ Learned result streaming for real-time processing
- ✅ Configured production monitoring and tuning parameters

**Key Takeaways**:
1. **Fan-out/fan-in** maximizes throughput by distributing work across concurrent workers and collecting results efficiently
2. **Poison pill pattern** (sending `None`) is the standard way to signal graceful shutdown to workers
3. **Queue sizing matters**: `work_queue_size` controls backpressure, `result_queue_size` prevents worker blocking
4. **Statistics are essential**: Per-worker metrics reveal load imbalance and performance issues
5. **Streaming results** enables real-time processing and reduces memory usage for large workloads

**When to Use This Pattern**:
- ✅ Batch processing with parallelizable tasks (image processing, data transformation, API calls)
- ✅ High-throughput pipelines (ETL, data ingestion, message processing)
- ✅ Background job processing (notification sending, report generation)
- ❌ Tasks with complex dependencies (use DAG scheduler like Airflow instead)
- ❌ Real-time request/response (use connection pooling or `bounded_map()` instead)

## Related Resources

**lionherd-core API Reference**:
- [Queue](../../docs/api/libs/concurrency/primitives.md) - Async FIFO queue with backpressure
- [TaskGroup](../../docs/api/libs/concurrency/task.md) - Structured concurrency for worker lifecycle
- [PriorityQueue](../../docs/api/libs/concurrency/priority_queue.md) - Priority-based work distribution

**Reference Notebooks**:
- [Concurrency Primitives](../references/concurrency_primitives.ipynb) - Deep dive into Queue, Lock, Event

**Related Tutorials**:
- [Deadline-Aware Task Queue](./deadline_task_queue.ipynb) - Queue processing with time budgets
- [Parallel Operations with Timeouts](./parallel_timeout.ipynb) - `bounded_map()` for simpler parallelism

**External Resources**:
- [Python asyncio Queues](https://docs.python.org/3/library/asyncio-queue.html) - Standard library queue patterns
- [Structured Concurrency](https://vorpus.org/blog/notes-on-structured-concurrency-or-go-statement-considered-harmful/) - Design principles behind TaskGroup
- [Producer-Consumer Pattern](https://en.wikipedia.org/wiki/Producer%E2%80%93consumer_problem) - Classic concurrency pattern